### Degradation問題
Degradation問題とは層が深くなれば深くなるほど表現力が上がってくるが、学習フェーズでなぜか浅い層に負ける。というように層を深くしたとしても必ずしも訓練誤差が下がっていくというわけではない。<br>
Microsoft社は勾配も消失していないと確認はしたけれども56層でできたニューラルネットワークは20層でできたニューラルネットワークには勝てないといった問題があった。

### 勾配消失が起きる理由
　DLで学習する際には各層ごとに活性化関数を微分して勾配を求める。まず入が層に近い『浅い』層ではその層への入力と出力の差(accuracy)が大きいため、微分操作は有効に働く。しかし、学習が層の『深い』段階へと進むにつれ、（当たり前だが）学習が収束に近づくと入力から出力への変換精度がどんどん上がっていくため、入力と出力の差は極めて小さくなり、勾配を取りにくくなる。層から層への伝播は掛け算の性質をもっているため、この差の減少傾向は指数関数的になる。つまり、層を増やしていくと、あっという間に勾配が消えてなくなってしまうのである。(http://terada-h.hatenablog.com/entry/2016/12/13/192940#:~:text=Residual%20Learning%EF%BC%88%E6%AE%8B%E5%B7%AE%E5%AD%A6%E7%BF%92,%E5%AD%A6%E7%BF%92%E3%81%99%E3%82%8B%E5%BD%A2%E3%81%AB%E3%81%99%E3%82%8B%E3%80%82)<br>
要は誤差逆伝播法でどんどんチェーンルールによって入力層に向かって更新していくのだが、深くしすぎることによってだんだんと反映されなくなってしまう。

### では深い層をどのようにすれば浅いモデルと同じような制度を生み出すことが出来るのか？？

→深い層を恒等関数にしてしまえば少なくとも浅いモデルと同じ精度になる。(udemy pytorch講座の28.ResNet概要5分ぐらいのところを見るとよい！)残差を学習していくという感じ<br>
入力をそのままスキップさせてしまうようにすれば実現可能（スキップコネクション）<br>
またミニバッチの中で正規化を行っていくと過学習しにくく学習が安定していく。（BatchNormalization）（バッチごとに平均と分散を計算していき、正規化を行っていく。udemyの
28．7分のところを確認）<br>
BatchNormalizationに関して(https://deepage.net/deep_learning/2016/10/26/batch_normalization.html)基本的には、勾配消失・爆発を防ぐための手法


In [1]:
import torch 
import torch.nn as nn

In [2]:
class ResidualBlock(nn.Module):
    #変数定義のところで変数を定義し
    #ニューラルネットはdef forwardのところで定義していく。
    def __init__(self , in_channels , out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels , out_channels , kernel_size = 3 , padding=1)
        self.conv2 = nn.Conv2d(out_channels , out_channels , kernel_size=3 , padding = 1)
        self.conv3 = nn.Conv2d(in_channels , out_channels , kernel_size=1 , padding=0)#あるチャネル数を持っているものを別のチャネル数に変更するというもの。
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    #出力されたものに対して入力値を足す際にチャネル数が違うのでチャネル数を合わせるための関数を定義している。
    def shortcut(self , x):
        x = self.conv3(x)#conv3でチャネル数がin_channelからout_channelに変更を行うことが出来る。
        x = self.bn(x)
        return x

    #残差ブロックの作成を行っている。
    def forward(self , x):
        identity = x #そのまま入力した値を代入している。
        x = self.conv1(x)
        x = self.bn(x)#Batch Normalizationを行っている。これを行うことによって学習の収束速度が上昇し、正則化の効果があったりしていいことが多かったりする
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn(x)
        x += self.shortcut(identity) #出力されたものに対して入力したものを足している。
        return x
    

In [5]:
class ResNet(nn.Module):
    def __init__(self , block) :#blockには先ほど定義した ResidualBlockを渡す。
        super().__init__()
        self.linear = nn.Linear(28*28*64 , out_features=10)#28*28の画像が64チャネルが入力される。10クラスの分類を行うと仮定して行っている。
        # _make_layerは下で定義している。
        self.layer = self._make_layer(block , 3 ,3, 64)#ブロックを3つ積み上げ、カラー画像を入力するとしてin_channelsは3,out_channelsは任意で指定し今回は64

    #ブロックをどんどん積み上げていく関数
    def _make_layer(self , block , num_residual_blocks , in_channels , out_channels ):#num_residual_blocksにはresidual_blockを何層作成するのかを入力する。
        layers = []
        for i in range(num_residual_blocks):
            if i == 0:#1層目に関してはチャネル数が画像のチャネル数なので分岐する必要がある
                layers.append(block(in_channels , out_channels))
            else:
                layers.append(block(out_channels , out_channels))
        return nn.Sequential(*layers)#複数のブロックを積み重ねたそうを出力している。

    def forward(self , x):
        x = self.layer(x)
        x = x.view(x.size(0) , -1)#Linearの層にデータを入れるため、データを一次元に変換している。
        x = self.linear(x)
        return x



            



In [6]:
#モデルをインスタンス化
model = ResNet(ResidualBlock)

In [7]:
model

ResNet(
  (linear): Linear(in_features=50176, out_features=10, bias=True)
  (layer): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1,

In [8]:
x_test = torch.randn(32 , 3 , 28 , 28)#バッチ数、チャネル数、縦＊横の大きさ

In [9]:
out_put = model(x_test)

In [10]:
out_put.size()#ミニバッチサイズで10クラスになっていることが確認できる。

torch.Size([32, 10])